# Encoder-decoder scaled dot-product attention

Recordamos la arquitectura del `Scaled Dot-Product Attention`

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention.png" alt="Scaled_Dot-Product_Attention">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Ya lo explicamos, pero ahora lo volvemos a explicar teniendo en cuenta que `K` y `V` es la matriz que proviene del encoder y `Q` proviene del decoder, por lo que se realizará una atención entre el encoder y el decoder

# MatMul

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_decoder_multi_head_attention.png" alt="Multi-Head Attention" style="width:425px;height:626px;">
</div>

Como ahora tenemos que `K` y `V` provienen del encoder, las llamaré $X_E$ y como `Q` proviene del decoder la llamaré $X_D$

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_first_MatMul.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Veamos cómo serían sus matrices, en primer lugar `K` y `V` o $X_E$

$$K=V=X_E = \begin{pmatrix}
v_{E,1} \\
v_{E,2} \\
\vdots\\
v_{E,m} \\
\end{pmatrix} = \begin{pmatrix}
v_{E,1,1} & v_{E,1,2} & \cdots & v_{E,1,n} \\
v_{E,2,1} & v_{E,2,2} & \cdots & v_{E,2,n} \\
\vdots & \vdots & \ddots & \vdots \\
v_{E,m,1} & v_{E,m,2} & \cdots & v_{E,m,n} \\
\end{pmatrix}$$

Y ahora `Q` o $X_D$

$$K=V=X_D = \begin{pmatrix}
v_{D,1} \\
v_{D,2} \\
\vdots\\
v_{D,m} \\
\end{pmatrix} = \begin{pmatrix}
v_{D,1,1} & v_{D,1,2} & \cdots & v_{D,1,n} \\
v_{D,2,1} & v_{D,2,2} & \cdots & v_{D,2,n} \\
\vdots & \vdots & \ddots & \vdots \\
v_{D,m,1} & v_{D,m,2} & \cdots & v_{D,m,n} \\
\end{pmatrix}$$

Por lo que la multiplicación entre `Q` y `K`, es decir, entre $X_E$ y $X_D$ es

$$X_D \cdot X_E^T = \begin{pmatrix}
v_{D,1} \cdot v_{E,1} & v_{D,1} \cdot v_{E,2} & \cdots & v_{D,1} \cdot v_{E,m} \\
v_{D,2} \cdot v_{E,1} & v_{D,2} \cdot v_{E,2} & \cdots & v_{D,2} \cdot v_{E,m} \\
\vdots & \vdots & \ddots & \vdots \\
v_{D,m} \cdot v_{E,1} & v_{D,m} \cdot v_{E,2} & \cdots & v_{D,m} \cdot v_{E,m} \\
\end{pmatrix}$$

La multiplicación sera una multiplicación de matrices de dimensiones $\left(m_D \times n_D\right) \cdot \left(n_E \times m_E\right)$, por lo que para que se pueda producir $n_D = n_E$, donde $n_D$ es la dimensión del embedding del decoder y $n_E$ es la dimensión del embedding del encoder. Es decir, para poder realizar esta operación, tanto el embedding del encoder como del decoder tienen que tener la misma dimensión.

Si ambas dimensiones de embedding son iguales, obtendremos como resultado una matriz de tamaño $\left(m_D \times m_E\right)$ donde $m_D$ era el número de tokens de la frase del decoder y $m_E$ era el número de tokens de la frase del encoder

Representamos la dimensión

$$X_D \cdot X_E^T = \begin{pmatrix}
v_{D,1} \cdot v_{E,1} & v_{D,1} \cdot v_{E,2} & \cdots & v_{D,1} \cdot v_{E,m} \\
v_{D,2} \cdot v_{E,1} & v_{D,2} \cdot v_{E,2} & \cdots & v_{D,2} \cdot v_{E,m} \\
\vdots & \vdots & \ddots & \vdots \\
v_{D,m} \cdot v_{E,1} & v_{D,m} \cdot v_{E,2} & \cdots & v_{D,m} \cdot v_{E,m} \\
\end{pmatrix}_{\left(m_D \times m_E\right)}$$

## Scale

A continuación se divide entre la dimensión del embedding de `K`, es decir, del encoder, pero en realidad da igual, porque hemos visto que la dimnesión del embedding del encoder y del decoder tienen que ser iguales. Esto se hace por ser una normalización `norma L2`

Así que nos queda

$$
\text{Scale} = \frac{1}{\sqrt{d_k}} \cdot \left( X_D \cdot X_E^T \right) = \frac{1}{\sqrt{d_k}} \cdot \begin{pmatrix}
v_{D,1} \cdot v_{E,1} & v_{D,1} \cdot v_{E,2} & \cdots & v_{D,1} \cdot v_{E,m} \\
v_{D,2} \cdot v_{E,1} & v_{D,2} \cdot v_{E,2} & \cdots & v_{D,2} \cdot v_{E,m} \\
\vdots & \vdots & \ddots & \vdots \\
v_{D,m} \cdot v_{E,1} & v_{D,m} \cdot v_{E,2} & \cdots & v_{D,m} \cdot v_{E,m} \\
\end{pmatrix}_{\left(m_D \times m_E\right)}
$$

## Mask

En este módulo de `Multi-Head Attention` no se realiza enmascaramiento, ya que el enmascaramiento se realiza para enmascarar el futuro de la secuencia de salida. Y eso ya lo hemos hecho en el anterior módulo de atención

## Softmax

Realizamos el `Softmax` de la amtriz que tenemos

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_softmax.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Por lo que nos quedaría una matriz así

$$
\text{Softmax} = \text{softmax}\left( \frac{1}{\sqrt{d_k}} \cdot \left( X_D \cdot X_E^T \right) \right) = \\
 = \text{softmax}\left( \frac{1}{\sqrt{d_k}} \cdot \begin{pmatrix}
v_{D,1} \cdot v_{E,1} & v_{D,1} \cdot v_{E,2} & \cdots & v_{D,1} \cdot v_{E,m} \\
v_{D,2} \cdot v_{E,1} & v_{D,2} \cdot v_{E,2} & \cdots & v_{D,2} \cdot v_{E,m} \\
\vdots & \vdots & \ddots & \vdots \\
v_{D,m} \cdot v_{E,1} & v_{D,m} \cdot v_{E,2} & \cdots & v_{D,m} \cdot v_{E,m} \\
\end{pmatrix}_{\left(m_D \times m_E\right)} \right)
$$

La cual podemos simplemente suponer como porcentajes de atención de los tokens salientes del encoder con los tokens del decoder

$$
\text{Softmax} = \begin{pmatrix}
p_{1D,1E} & p_{1D,2E} & \cdots & p_{1D,mE} \\
p_{2D,1E} & p_{2D,2E} & \cdots & p_{2D,mE} \\
\vdots & \vdots & \ddots & \vdots \\
p_{mD,1E} & p_{mD,2E} & \cdots & p_{mD,mE} \\
\end{pmatrix}_{\left(m_D \times m_E\right)}
$$

Por ejemplo en la traducción de `¿Cuál es ti nombre?` a `What is your name` el porcentaje de atención entre `nombre` y `name` debería ser muy alto, pues esta matriz representa la atención entre los tokens de la frase en español con los tokens de la frase en ingles

## MatMul

Por último volvemos a realizar un `MatMul` entre la matriz que tenemos que es de dimensiones $\left(m_D \times m_E\right)$ por `V` que como hemos dicho proviene de la salida del encoder ($X_E$), por lo que tiene dimensiones $\left(m_E \times n_E\right)$. Así que la multiplicación va a ser de dimensiones $\left(m_D \times m_E\right)·\left(m_E \times n_E\right)$, lo que nos da una matriz de $\left(m_D \times n_E\right)$, pero como las dimensiones del embedding del encoder y del decoder tienen que ser iguales $\left(n_D = n_E\right)$ nos queda una matriz de tamaño $\left(m_D \times n_D\right)$

Esto nos hace ver que hemos hecho lo correcto, porque si recuerdas, en el encoder siempre trabajamos con matrices $\left(m_E \times n_E\right)$, en el módulo de atención del encoder entraba una matriz de tamaño $\left(m_E \times n_E\right)$ y salía una matriz de tamaño $\left(m_E \times n_E\right)$. Incluso en el encoder entero entraba una matriz de tamaño $\left(m_E \times n_E\right)$ y salía una matriz de tamaño $\left(m_E \times n_E\right)$.

Por lo que ahora en si al decoder le entra una matriz $\left(m_D \times n_D\right)$ y a la salida de este segundo módulo de atención tenemos una matriz de tamaño $\left(m_D \times n_D\right)$ es que hemos hecho bien las cosas

Si realizamos la operación

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_second_MatMul.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

En realidad lo que tenemos es la siguiente matriz

$$
\text{Matmul} = \begin{pmatrix}
p_{1D,1E} & p_{1D,2E} & \cdots & p_{1D,mE} \\
p_{2D,1E} & p_{2D,2E} & \cdots & p_{2D,mE} \\
\vdots & \vdots & \ddots & \vdots \\
p_{mD,1E} & p_{mD,2E} & \cdots & p_{mD,mE} \\
\end{pmatrix}_{\left(m_D \times m_E\right)} \cdot \begin{pmatrix}
v_{E,1} \\
v_{E,2} \\
\vdots\\
v_{E,m} \\
\end{pmatrix} = \\
 = \begin{pmatrix}
p_{1D,1E}·v_{E,1} + p_{1D,2E}·v_{E,2} + \cdots + p_{1D,mE}·v_{E,m} \\
p_{2D,1E}·v_{E,1} + p_{2D,2E}·v_{E,2} + \cdots + p_{2D,mE}·v_{E,m} \\
\vdots \\
p_{mD,1E}·v_{E,1} + p_{mD,2E}·v_{E,2} + \cdots + p_{mD,mE}·v_{E,m} \\
\end{pmatrix}
$$

Que representa

 * La primera fila (que representaría el primer token) se corresponde a la suma de probabilidades de atención del primer token del decoder con el resto de tokens del encoder por los embeddings del resto de tokens del encoder
 * La segunda fila (que representaría al segundo token) se corresponde a la suma de probabilidades de atención del segundo token del decoder con el resto de tokens del encoder por los embeddings del resto de tokens del encoder
 * Así sucesivamente, hasta la última fila (que representaría al último token) que se corresponde a la suma de probabilidades del último token del decoder con el resto de tokens del encoder por los embeddings del resto de tokens del encoder

# Implementación

La clase que hemos hecho hasta ahora para el `Scaled Dot-Product Attention` nos vale, ya que las operaciones son válidas, solo que tendremos que tener en cuenta que cuando la usemos, en este caso `K` y `V` tendrán que ser la matriz que sale del encoder y `Q` la matriz del decoder

In [1]:
import torch
import torch.nn as nn

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, key, query, value, mask=None):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
            mask: mask matrix (optional)
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        scale = product / torch.sqrt(torch.tensor(self.dim_embedding))
        # Mask (optional)
        if mask is not None:
            scale = scale.masked_fill(mask == 0, float('-inf'))
        # softmax
        attention_matrix = torch.softmax(scale, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output

Como vamos a necesitar la salida del encoder volvemos a escribir todas las clases del encoder

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Embedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        """
        Args:
            vocab_size: size of vocabulary
            embed_dim: dimension of embeddings
        """
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            out: embedding vector
        """
        return self.embedding(x)

class PositionalEncoding(nn.Module):
    def __init__(self, max_sequence_len, embedding_model_dim):
        """
        Args:
            seq_len: length of input sequence
            embed_model_dim: demension of embedding
        """
        super().__init__()
        self.embedding_dim = embedding_model_dim

        # create constant 'positional_encoding' matrix with values dependant on pos and i
        positional_encoding = torch.zeros(max_sequence_len, self.embedding_dim)
        for pos in range(max_sequence_len):
            for i in range(0, self.embedding_dim, 2):
                positional_encoding[pos, i]     = torch.sin(torch.tensor(pos / (10000 ** ((2 * i) / self.embedding_dim))))
                positional_encoding[pos, i + 1] = torch.cos(torch.tensor(pos / (10000 ** ((2 * (i+1)) / self.embedding_dim))))
        positional_encoding = positional_encoding.unsqueeze(0)
        self.register_buffer('positional_encoding', positional_encoding)

    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            x: output
        """
        # make embeddings relatively larger
        x = x * torch.sqrt(torch.tensor(self.embedding_dim))
        
        # add encoding matrix to embedding (x)
        sequence_len = x.size(1)
        # x = x + torch.autograd.Variable(self.positional_encoding[:,:sequence_len], requires_grad=False)
        x = x + self.positional_encoding[:,:sequence_len]
        return x

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, query, key, value, mask=None):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
            mask: mask matrix (optional)
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        scale = product / torch.sqrt(torch.tensor(self.dim_embedding))
        # Mask (optional)
        if mask is not None:
            scale = scale.masked_fill(mask == 0, float('-inf'))
        # softmax
        attention_matrix = torch.softmax(scale, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, dim_embedding):
        """
        Args:
            heads: number of heads
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        
        self.dim_embedding = dim_embedding
        self.dim_proyection = dim_embedding // heads
        self.heads = heads
        
        self.proyection_Q = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_K = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_V = nn.Linear(dim_embedding, dim_embedding)
        self.attention = nn.Linear(dim_embedding, dim_embedding)

        self.scaled_dot_product_attention = ScaledDotProductAttention(self.dim_proyection)
    
    def forward(self, Q, K, V, mask=None):
        """
        Args:
            Q: query vector
            K: key vector
            V: value vector
            mask: mask matrix (optional)

        Returns:
            output vector from multi-head attention
        """
        batch_size = Q.size(0)
        
        # perform linear operation and split into h heads
        proyection_Q = self.proyection_Q(Q).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_K = self.proyection_K(K).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_V = self.proyection_V(V).view(batch_size, -1, self.heads, self.dim_proyection)
        
        # transpose to get dimensions bs * h * sl * d_model
        proyection_Q = proyection_Q.transpose(1,2)
        proyection_K = proyection_K.transpose(1,2)
        proyection_V = proyection_V.transpose(1,2)

        # calculate attention
        scaled_dot_product_attention = self.scaled_dot_product_attention(proyection_Q, proyection_K, proyection_V, mask=mask)
        
        # concatenate heads and put through final linear layer
        concat = scaled_dot_product_attention.transpose(1,2).contiguous().view(batch_size, -1, self.dim_embedding)
        
        output = self.attention(concat)
    
        return output

class AddAndNorm(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding (int): Embedding dimension.
        """
        super().__init__()
        self.normalization = nn.LayerNorm(dim_embedding)

    def forward(self, x, sublayer):
        """
        Args:
            x (torch.Tensor): Input tensor.
            sublayer (torch.Tensor): Sublayer tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        return self.normalization(torch.add(x, sublayer))

class FeedForward(nn.Module):
    def __init__(self, dim_embedding, increment=4):
        super().__init__()
        self.feed_forward = nn.Sequential(
            nn.Linear(dim_embedding, dim_embedding*increment),
            nn.ReLU(),
            nn.Linear(dim_embedding*increment, dim_embedding)
        )
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        x = self.feed_forward(x)
        return x

class EncoderLayer(nn.Module):
    def __init__(self, heads, dim_embedding):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(heads, dim_embedding)
        self.add_and_norm_1 = AddAndNorm(dim_embedding)
        self.feed_forward = FeedForward(dim_embedding)
        self.add_and_norm_2 = AddAndNorm(dim_embedding)
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        multi_head_attention = self.multi_head_attention(x, x, x)
        add_and_norm_1 = self.add_and_norm_1(x, multi_head_attention)
        feed_forward = self.feed_forward(add_and_norm_1)
        add_and_norm_2 = self.add_and_norm_2(add_and_norm_1, feed_forward)
        return add_and_norm_2

class Encoder(nn.Module):
    def __init__(self, heads, dim_embedding, Nx):
        super().__init__()
        self.encoder_layers = nn.ModuleList([EncoderLayer(heads, dim_embedding) for _ in range(Nx)])
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x)
        return x

class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, dim_embedding, max_sequence_len, heads, Nx):
        super().__init__()
        self.input_embedding = Embedding(vocab_size, dim_embedding)
        self.positional_encoding = PositionalEncoding(max_sequence_len, dim_embedding)
        self.encoder = Encoder(heads, dim_embedding, Nx)
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        input_embedding = self.input_embedding(x)
        positional_encoding = self.positional_encoding(input_embedding)
        encoder = self.encoder(positional_encoding)
        return encoder


Volvemos a definir la función que obtiene el embbeding más el positional encoding de BERT. Ahora no usaremos su embedding, pero sí sus tokens

In [3]:
import torch
from transformers import BertModel, BertTokenizer

def extract_embeddings(input_sentences, model_name='bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    
    # tokenización de lote
    inputs = tokenizer(input_sentences, return_tensors='pt', padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
        
    token_embeddings = outputs[0]
    
    # Los embeddings posicionales están en la segunda capa de los embeddings de la arquitectura BERT
    positional_encodings = model.embeddings.position_embeddings.weight[:token_embeddings.shape[1], :].detach().unsqueeze(0).repeat(token_embeddings.shape[0], 1, 1)

    embeddings_with_positional_encoding = token_embeddings + positional_encodings

    # convierte las IDs de los tokens a tokens
    tokens = [tokenizer.convert_ids_to_tokens(input_id) for input_id in inputs['input_ids']]

    return tokens, inputs['input_ids'], token_embeddings, positional_encodings, embeddings_with_positional_encoding

Creamos una sentencia para el encoder, ya que ahora va a entrar una sentencia al encoder y otra al decoder

In [4]:
sentence_encoder = "I gave the dog a bone because it was hungry"
tokens_encoder, input_ids_encoder, token_embeddings_encoder, positional_encodings_encoder, embeddings_with_positional_encoding_encoder = extract_embeddings(sentence_encoder)

Creamos un objeto `encoder`

In [5]:
vocab_size = 30522
dim_embedding = token_embeddings_encoder.shape[-1]
max_sequence_len = token_embeddings_encoder.shape[1]
heads = 8
Nx = 6
transformer_encoder = TransformerEncoder(vocab_size, dim_embedding, max_sequence_len, heads, Nx)

Obtenemos la salida del transformer encoder

In [6]:
encoder_output = transformer_encoder(input_ids_encoder)
encoder_output.shape

torch.Size([1, 12, 768])

Ahora generamos una sentencia para el decoder, lo que haremos será generar la sentencia del encoder traducida al español

In [7]:
sentence_encoder = "I gave the dog a bone because it was hungry"
sentence_decoder = "Le di un hueso al perro porque tenía hambre"
tokens_decoder, input_ids_decoder, token_embeddings_decoder, positional_encodings_decoder, embeddings_with_positional_encoding_decoder = extract_embeddings(sentence_decoder)

Vamos a ver cómo es la secuencia del encoder y del decoder

In [8]:
print(f"Numero de tokens del encoder: {len(tokens_encoder[0])}, embedding encoder shape: {token_embeddings_encoder.shape}, positional encodings encoder shape: {positional_encodings_encoder.shape}, embeddings with positional encodings encoder shape: {embeddings_with_positional_encoding_encoder.shape}")
print(f"Numero de tokens del decoder: {len(tokens_decoder[0])}, embedding decoder shape: {token_embeddings_decoder.shape}, positional encodings decoder shape: {positional_encodings_decoder.shape}, embeddings with positional encodings decoder shape: {embeddings_with_positional_encoding_decoder.shape}")

Numero de tokens del encoder: 12, embedding encoder shape: torch.Size([1, 12, 768]), positional encodings encoder shape: torch.Size([1, 12, 768]), embeddings with positional encodings encoder shape: torch.Size([1, 12, 768])
Numero de tokens del decoder: 16, embedding decoder shape: torch.Size([1, 16, 768]), positional encodings decoder shape: torch.Size([1, 16, 768]), embeddings with positional encodings decoder shape: torch.Size([1, 16, 768])


Como vemos la frase en inglés (`encoder`) tiene 12 tokens y en español (`decoder`) tiene 16 tokens

Instanciamos un objeto de la clase `Multi-Head Attention` enmascarada y obtenemos su salida

In [9]:
def create_mask(sequence_len):
    """
    Args:
        sequence_len: length of sequence
        
    Returns:
        mask matrix
    """
    mask = torch.tril(torch.ones((sequence_len, sequence_len)))
    return mask
sequence_len = input_ids_decoder.shape[1]
mask = create_mask(sequence_len)
mask.shape

torch.Size([16, 16])

In [10]:
dim_embedding = embeddings_with_positional_encoding_decoder.shape[-1]
heads = 8
masked_multi_head_attention = MultiHeadAttention(heads=heads, dim_embedding=dim_embedding)

K = embeddings_with_positional_encoding_decoder
V = embeddings_with_positional_encoding_decoder
Q = embeddings_with_positional_encoding_decoder
masked_attention = masked_multi_head_attention(Q, K, V, mask=mask)
masked_attention.shape

torch.Size([1, 16, 768])

Como hemos visto, la secuencia del decoder tenía 16 tokens, por lo que tiene sentido que la dimensión sea 1x16x768

Creamos ahora un objeto de la clase `Add & Norm` y calculamos su salida

In [11]:
add_and_norm_3 = AddAndNorm(dim_embedding)
masked_attention_add_and_norm = add_and_norm_3(embeddings_with_positional_encoding_decoder, masked_attention)
masked_attention_add_and_norm.shape

torch.Size([1, 16, 768])

Ya tenemos todas las entradas para el `Encoder-Decoder Scaled Dot-Product Attention`, así que creamos un objeto y obtenemos su salida

In [12]:
encoder_decoder_scaled_dot_product_attention = ScaledDotProductAttention(dim_embedding)
K = encoder_output
V = encoder_output
Q = masked_attention_add_and_norm
encoder_decoder_attention = encoder_decoder_scaled_dot_product_attention(Q, K, V)
encoder_decoder_attention.shape

torch.Size([1, 16, 768])

No obtenemos una matriz de 1x12x768 (dimensión del embedding del encoder) sino 1x16x768 que es la dimensión del embedding del decoder. Por lo que aunque las secuencias del encoder y del decoder no tengan el mismo número de tokens no hay problema, ya que el `Encoder-Decoder Scaled Dot-Product Attention` a la salida seguirá dando el número de tokens del decoder

Si lo pensamos, es lo que tiene que pasar, si el decoder tiene 16 tokens y el encoder 12, a la salida del `Encoder-Decoder Scaled Dot-Product Attention` tiene que generar una secuencia con el número de tokens del decoder, ya que queremos predecir el siguiente token del decoder

Vamos a ver matemáticamente por qué pasa eso, primero recordamos la arquirtectura y la fórmula del `Scaled Dot-Product Attention`

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention.png" alt="Scaled_Dot-Product_Attention">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Y segundo recordamos que `K` y `V` es la matriz que proviene del encoder y `Q` proviene del decoder

De modo que primero tenemos una multiplicación entre `Q` y la traspuesta de `K`, es decir tendremos una multiplicación de dimensiones $\left(m_D \times n_D\right) \cdot \left(n_E \times m_E\right)$, donde $m_D$ y $m_E$ son el número de tokens del decoder y encoder respectivamente y $n_D$ y $n_E$ son la dimensión del embedding del decoder y encoder respectivamente y que para poder hacer la multiplicación de las matrices tienen que ser iguales. De modo que nos queda una matriz de dimensiones $\left(m_D \times m_E\right)$

Luego la operación de `Scale` no cambia el tamaño de la matriz, aquí no enmascaramos y la operación de `Softmax` tampoco cambia el tamaño

Así que al final nos quedamos con la multiplicación de una matriz de tamaño $\left(m_D \times m_E\right)$ por otra de tamaño $\left(m_E \times n_E\right)$, es decir $\left(m_D \times m_E\right) \cdot \left(m_E \times n_E\right)$, por lo que nos queda una matriz de tamaño $\left(m_D \times n_E\right)$, pero como hemos dicho que la dimensión del embedding del encoder y del decoder tiene que ser la misma, entonces queda $\left(m_D \times n\right)$

Es decir, nos queda una matriz con el número de tokens del decoder